## Project - Neural Networks - Project - Part 3

In [1]:
# Importing required libraries

import tkinter as tk
from tkinter import ttk
import pandas as pd
import re


# App window
win = tk.Tk()
# Window Title
win.title('Neural Networks GUI - Great Learning') 

# Importing data frame name

Name=ttk.Label(win,text="    Step 1: File Name")
Name.grid(row=0,column=0,sticky=tk.W)

Name_var=tk.StringVar()
Name_entrybox=ttk.Entry(win,width=16,textvariable=Name_var)
Name_entrybox.grid(row=0,column=1)

def Import_Data():
    global DB
    DF_Name=Name_var.get()
    DB_extension=re.findall("\..*", DF_Name) 
    if DB_extension==['.xlsx']:
        DB=pd.read_excel(DF_Name)
    elif DB_extension==['.csv']:
        DB=pd.read_csv(DF_Name)
    # Blank empty window to print confirmation
    confirm="Done"
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=0,column=3)
    Confirm_entrybox.insert(1,str(confirm))   

Import_Data_Button=ttk.Button(win,text="Import Data",command=Import_Data)
Import_Data_Button.grid(row=0,column=2)


# Target data frame name

Target=ttk.Label(win,text="    Step 2: Target Colummn")
Target.grid(row=1,column=0,sticky=tk.W)

Target_var=tk.StringVar()
Target_entrybox=ttk.Entry(win,width=16,textvariable=Target_var)
Target_entrybox.grid(row=1,column=1)

def Target_Data():
    global DB,X,y, Target_Name, X_train, X_test, y_train, y_test
    Target_Name=Target_var.get()
    
    Column_name=DB.columns
    Column_name
    found=0

    for i in range(len(Column_name)):
        if Column_name[i]==Target_Name:
            confirm="Found"
        else:
            confirm="Not Found"
    
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=1,column=3)
    Confirm_entrybox.insert(1,str(confirm))


Target_Button=ttk.Button(win,text="Import Target",command=Target_Data)
Target_Button.grid(row=1,column=2)


# Regression

Modelling=ttk.Label(win,text="    Step 3: Neural Network Regressor")
Modelling.grid(row=2,column=0,sticky=tk.W)

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential # Forward prop
from keras.layers import Dense, Activation, LeakyReLU
from keras import optimizers

Regression=ttk.Label(win,text="Regression")
Regression.grid(row=3,column=0,sticky=tk.E)


def NNReg():
    global DB,X,Y,NN_model_Regressor
    
    # Regression
    
    # Independent variables
    X=DB.drop('Signal_Strength',axis=1)
    
    # Target variable
    Y=DB['Signal_Strength']               

    X_Train,X_Test,Y_Train,Y_Test=train_test_split(X, Y, train_size=0.7, random_state=12)
    
    X_Train_S = StandardScaler().fit_transform(X_Train)   # Scale train data
    X_Test_S = StandardScaler().fit_transform(X_Test)     # Scale test data
    
    # Model 
    NN_model_Regressor = Sequential()
    NN_model_Regressor.add(Dense(128, kernel_initializer='normal',input_dim = X_Train.shape[1], activation='relu'))
    NN_model_Regressor.add(Dense(64, kernel_initializer='normal',activation='relu'))  # sigmoid, tanh
    NN_model_Regressor.add(Dense(32, kernel_initializer='normal'))
    NN_model_Regressor.add(LeakyReLU(alpha=0.1))
    NN_model_Regressor.add(Dense(16, kernel_initializer='normal'))
    NN_model_Regressor.add(LeakyReLU(alpha=0.1))
    NN_model_Regressor.add(Dense(1, kernel_initializer='normal'))  # except softmax
    NN_model_Regressor.add(LeakyReLU(alpha=0.1))
    
    # Compile the network :
    NN_model_Regressor.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])
    NN_model_Regressor.summary()
    EPOCH=400
    Network_Regressor=NN_model_Regressor.fit(X_Train_S, Y_Train, validation_data=(X_Test_S,Y_Test), epochs=EPOCH, batch_size=200)
    
    
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=3,column=2)
    Confirm_entrybox.insert(1,str("Network Trained"))

Reg_Button=ttk.Button(win,text="Train",command=NNReg)
Reg_Button.grid(row=3,column=1)

from keras.models import model_from_json
import numpy
import os

Pickle=ttk.Label(win,text="Pickle")
Pickle.grid(row=4,column=0,sticky=tk.E)

def Pickle():
    # Pickle model to JSON
    Regressor_model_json = NN_model_Regressor.to_json()
    with open("Regressor_model.json", "w") as json_file:
        json_file.write(Regressor_model_json)
    # Pickle weights to HDF5
    NN_model_Regressor.save_weights("Regressor_model.h5")
    
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=4,column=2)
    Confirm_entrybox.insert(1,str("Saved model to disk"))

Pickle_Button=ttk.Button(win,text="Run",command=Pickle)
Pickle_Button.grid(row=4,column=1)


# Classifier

Modelling=ttk.Label(win,text="    Step 4: Neural Network Classifier")
Modelling.grid(row=5,column=0,sticky=tk.W)

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils.np_utils import to_categorical
from keras.models import Sequential # Forward prop
from keras.layers import Dense, Activation, LeakyReLU
from keras import optimizers
from keras.models import model_from_json
import numpy
import os

Classifier=ttk.Label(win,text="Classifier")
Classifier.grid(row=6,column=0,sticky=tk.E)

def NNClassi():
    global DB,X,Y,NN_model_Classifier
    
    # Regression
    
    # Independent variables
    X=DB.drop('Signal_Strength',axis=1)
    
    # Target variable
    Y=DB['Signal_Strength']               

    X_Train,X_Test,Y_Train,Y_Test=train_test_split(X, Y, train_size=0.7, random_state=12)

    X_Train,X_Test,Y_Train,Y_Test=train_test_split(X, Y, train_size=0.7, random_state=12)
    
    X_Train_S = StandardScaler().fit_transform(X_Train)   # Scale train data
    X_Test_S = StandardScaler().fit_transform(X_Test)     # Scale test data
    
    Y_Train = to_categorical(Y_Train)
    Y_Test = to_categorical(Y_Test)
    
    NN_model_Classifier = Sequential()

    # The Input Layer :
    NN_model_Classifier.add(Dense(128, kernel_initializer='normal',input_dim = X_Train.shape[1], activation='relu'))

    # The Hidden Layers :
    NN_model_Classifier.add(Dense(64, kernel_initializer='normal',activation='relu'))  # sigmoid, tanh

    NN_model_Classifier.add(Dense(32, kernel_initializer='normal'))
    NN_model_Classifier.add(LeakyReLU(alpha=0.1))

    NN_model_Classifier.add(Dense(16, kernel_initializer='normal'))
    NN_model_Classifier.add(LeakyReLU(alpha=0.1))


    # The Output Layer :
    NN_model_Classifier.add(Dense(9, kernel_initializer='normal',activation='softmax'))  # except softmax

    # Compile the network :
    NN_model_Classifier.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])
    NN_model_Classifier.summary()
    EPOCH=400
    Network_Classifier=NN_model_Classifier.fit(X_Train_S, Y_Train, validation_data=(X_Test_S,Y_Test), epochs=EPOCH, batch_size=200)
    
    
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=6,column=2)
    Confirm_entrybox.insert(1,str("Network Trained"))

Cla_Button=ttk.Button(win,text="Train",command=NNClassi)
Cla_Button.grid(row=6,column=1)

Pickle=ttk.Label(win,text="Pickle")
Pickle.grid(row=4,column=0,sticky=tk.E)

def Pickle2():
    # Pickle model to JSON
    Classifier_model_json = NN_model_Classifier.to_json()
    with open("Classifier_model.json", "w") as json_file:
        json_file.write(Classifier_model_json)
    # Pickle weights to HDF5
    NN_model_Classifier.save_weights("Classifier_model.h5")
    
    Confirm_entrybox=ttk.Entry(win,width=16)
    Confirm_entrybox.grid(row=7,column=2)
    Confirm_entrybox.insert(1,str("Saved model to disk"))

Pickle_Button2=ttk.Button(win,text="Run",command=Pickle2)
Pickle_Button2.grid(row=7,column=1)



win.mainloop()

Using TensorFlow backend.
/Users/krishnavdave/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/krishnavdave/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/krishnavdave/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/krishnavdave/opt





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               1536      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                528       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
______

1119/1119 [==============================] - 0s - loss: 0.4757 - acc: 0.6122 - val_loss: 0.4814 - val_acc: 0.5958
Epoch 105/400
1119/1119 [==============================] - 0s - loss: 0.4720 - acc: 0.6139 - val_loss: 0.4889 - val_acc: 0.6063
Epoch 106/400
1119/1119 [==============================] - 0s - loss: 0.4782 - acc: 0.6211 - val_loss: 0.4802 - val_acc: 0.6062
Epoch 107/400
1119/1119 [==============================] - 0s - loss: 0.4756 - acc: 0.6113 - val_loss: 0.4852 - val_acc: 0.6104
Epoch 108/400
1119/1119 [==============================] - 0s - loss: 0.4767 - acc: 0.6122 - val_loss: 0.4828 - val_acc: 0.6188
Epoch 109/400
1119/1119 [==============================] - 0s - loss: 0.4829 - acc: 0.6014 - val_loss: 0.4864 - val_acc: 0.6083
Epoch 110/400
1119/1119 [==============================] - 0s - loss: 0.4764 - acc: 0.6157 - val_loss: 0.4808 - val_acc: 0.6063
Epoch 111/400
1119/1119 [==============================] - 0s - loss: 0.4738 - acc: 0.6122 - val_loss: 0.4820 - val_ac

1119/1119 [==============================] - 0s - loss: 0.4160 - acc: 0.6595 - val_loss: 0.4649 - val_acc: 0.6250
Epoch 233/400
1119/1119 [==============================] - 0s - loss: 0.4172 - acc: 0.6649 - val_loss: 0.4587 - val_acc: 0.6271
Epoch 234/400
1119/1119 [==============================] - 0s - loss: 0.4191 - acc: 0.6649 - val_loss: 0.4572 - val_acc: 0.6354
Epoch 235/400
1119/1119 [==============================] - 0s - loss: 0.4244 - acc: 0.6550 - val_loss: 0.4560 - val_acc: 0.6312
Epoch 236/400
1119/1119 [==============================] - 0s - loss: 0.4214 - acc: 0.6667 - val_loss: 0.4545 - val_acc: 0.6333
Epoch 237/400
1119/1119 [==============================] - 0s - loss: 0.4160 - acc: 0.6604 - val_loss: 0.4549 - val_acc: 0.6354
Epoch 238/400
1119/1119 [==============================] - 0s - loss: 0.4147 - acc: 0.6595 - val_loss: 0.4677 - val_acc: 0.6125
Epoch 239/400
1119/1119 [==============================] - 0s - loss: 0.4188 - acc: 0.6577 - val_loss: 0.4542 - val_ac

1119/1119 [==============================] - 0s - loss: 0.3876 - acc: 0.7024 - val_loss: 0.4692 - val_acc: 0.6208
Epoch 361/400
1119/1119 [==============================] - 0s - loss: 0.3863 - acc: 0.7078 - val_loss: 0.4632 - val_acc: 0.6312
Epoch 362/400
1119/1119 [==============================] - 0s - loss: 0.3892 - acc: 0.7015 - val_loss: 0.4531 - val_acc: 0.6312
Epoch 363/400
1119/1119 [==============================] - 0s - loss: 0.3846 - acc: 0.7122 - val_loss: 0.4638 - val_acc: 0.6292
Epoch 364/400
1119/1119 [==============================] - 0s - loss: 0.3837 - acc: 0.7051 - val_loss: 0.4542 - val_acc: 0.6354
Epoch 365/400
1119/1119 [==============================] - 0s - loss: 0.3795 - acc: 0.7087 - val_loss: 0.4462 - val_acc: 0.6208
Epoch 366/400
1119/1119 [==============================] - 0s - loss: 0.3722 - acc: 0.7105 - val_loss: 0.4546 - val_acc: 0.6271
Epoch 367/400
1119/1119 [==============================] - 0s - loss: 0.3748 - acc: 0.7069 - val_loss: 0.4522 - val_ac